In [35]:
import timeit
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.sandbox.regression.gmm import IVGMM
from statsmodels.api import add_constant
import patsy as ps
import wooldridge

# Regressão Linear

#### 1) Um problema de interesse para autoridades de saúde é determinar os efeitos do fumo durante a gravidez na saúde do bebê. Uma medida da saúde do bebê é o peso no nascimento (bwght); um peso no nascimento muito baixo pode colocar o bebê em risco de contrair várias doenças. Uma vez que fatores além do tabagismo que afetam o peso no nascimento provavelmente estão correlacionados com o tabagismo, devemos levar esses fatores em consideração. Por exemplo, uma renda mais alta geralmente resulta em acesso a melhor atendimento pré-natal, bem como melhor nutrição para a mãe. Uma equação que reconhece isso é:

$$\text{bwght} = \beta_1 \text{cigs} + \beta_2 \text{faminc} + u$$

In [36]:
bwght = wooldridge.data('bwght')
bwght.head()

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,lbwght,bwghtlbs,packs,lfaminc
0,13.5,16.5,122.300003,109,12.0,12.0,1,1,1,0,4.691348,6.8125,0.0,2.602690
1,7.5,16.5,122.300003,133,6.0,12.0,2,1,0,0,4.890349,8.3125,0.0,2.014903
2,0.5,16.5,122.300003,129,NaN,12.0,2,0,0,0,4.859812,8.0625,0.0,-0.693147
3,15.5,16.5,122.300003,126,12.0,12.0,2,1,0,0,4.836282,7.8750,0.0,2.740840
4,27.5,16.5,122.300003,134,14.0,12.0,2,1,1,0,4.897840,8.3750,0.0,3.314186


##### (i) Qual é o sinal mais provável para $\beta_2$?

Podemos imaginar que $\beta_2 > 0$, pelo raciocínio a priori de que uma maior renda traria melhores cuidados para a mãe durante o pré-natal.

##### (ii) Você acha que cigs e faminc provavelmente estão correlacionados? Explique por que a correlação pode ser positiva ou negativa.

Por um lado, um aumento na renda geralmente aumenta o consumo de um bem, e cigs e faminc podem estar positivamente correlacionados. 
Por outro lado, os rendimentos familiares também são maiores para famílias com mais educação, e a educação e o consumo de cigarros tendem a estar negativamente correlacionados.

In [37]:
bwght.cigs.corr(bwght.faminc)

-0.1730449257358646

##### (iii) Agora, estime a equação com e sem faminc, usando os dados de BWGHT do Wooldridge. Relate os resultados em forma de equação, incluindo o tamanho da amostra e R-quadrado. Discuta seus resultados, focando se  inclusão de faminc muda substancialmente o efeito estimado de cigs em bwght.

In [38]:
# Regressão por OLS sem faminc
formula = 'bwght ~ cigs'
y, X = ps.dmatrices(formula, data=bwght, return_type='dataframe')
model = sm.OLS(y, X, missing="drop").fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     32.24
Date:                Tue, 20 Jun 2023   Prob (F-statistic):           1.66e-08
Time:                        17:12:28   Log-Likelihood:                -6135.5
No. Observations:                1388   AIC:                         1.227e+04
Df Residuals:                    1386   BIC:                         1.229e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    119.7719      0.572    209.267      0.0

In [39]:
# Regressão por OLS com faminc
formula = 'bwght ~ cigs + faminc'
y, X = ps.dmatrices(formula, data=bwght, return_type='dataframe')
model = sm.OLS(y, X, missing="drop").fit()
print(model.summary())

"""
O efeito do consumo de cigarros é ligeiramente menor quando faminc é adicionado 
à regressão, mas a diferença não é significativa. Isso ocorre devido ao fato de 
que cigs e faminc não estão muito correlacionados, e o coeficiente em faminc é 
pequeno. (A variável faminc é medida em milhares, então um aumento de $10.000 
na renda de 1988 aumenta o peso de nascimento previsto em apenas 0,93 onças.)
""";

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     21.27
Date:                Tue, 20 Jun 2023   Prob (F-statistic):           7.94e-10
Time:                        17:12:28   Log-Likelihood:                -6130.4
No. Observations:                1388   AIC:                         1.227e+04
Df Residuals:                    1385   BIC:                         1.228e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    116.9741      1.049    111.512      0.0

#### 2) Considere o seguinte modelo para explicar o comportamento do sono:

$$ \text{sleep} = \beta_0 + \beta_1 \text{totwork} + \beta_2 \text{educ} + \beta_3 \text{age} + \beta_4 \text{age}^2 + \beta_5 \text{yngkid} + \beta_6 \text{male} + u

##### (i) Escreva um modelo que permita que a variância de u difira entre homens e mulheres. A variância não deve depender de outros fatores.

No modelo pedido, a variância de u deve depender apenas do gênero. Portanto, temos:
$$Var(u|totwrk,educ,age,yngkid,male) = Var(u|male) = \delta_0 + \delta_1 male$$
A variância para mulheres será apenas $\delta_0$ e para homens $\delta_0 + \delta_1$. A diferença entre variâncias será $\delta_1$.

##### (ii) Use os dados em SLEEP75 (Wooldridge) para estimar os parâmetros do modelo para heteroscedasticidade. (Você precisa primeiro estimar a equação de sono por Mínimos Quadrados Ordinários (OLS) para obter os resíduos de OLS.) A variância estimada de u é maior para homens ou para mulheres? 

##### (iii) A variância de u é estatisticamente diferente para homens e mulheres?

In [40]:
sleep75 = wooldridge.data('sleep75')

In [41]:
# Estimando sono por OLS
formula = 'sleep ~ totwrk + educ + age + agesq + yngkid + male'
y, X = ps.dmatrices(formula, data=sleep75, return_type='dataframe')
model = sm.OLS(y, X, missing="drop").fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sleep   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     16.30
Date:                Tue, 20 Jun 2023   Prob (F-statistic):           1.28e-17
Time:                        17:12:28   Log-Likelihood:                -5259.3
No. Observations:                 706   AIC:                         1.053e+04
Df Residuals:                     699   BIC:                         1.056e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3840.8521    239.414     16.043      0.0

In [43]:
# Regredindo o resíduo ao quadrado em male
sleep75["u_sq"] = model.resid ** 2
formula = 'u_sq ~ male'
y, X = ps.dmatrices(formula, data=sleep75, return_type='dataframe')
model = sm.OLS(y, X, missing="drop").fit()
print(model.summary())

"""
Vemos que o intercepto para "male" na regressão com o resíduo quadrado como 
variável dependente é negativo, o que indicaria que a variância estimada é 
menor para homens.

Porém, o resultado de menor variância no erro para homens não é significante, 
pois temos uma estatística t de apenas -1.06 (p-valor 0.291: não é significante 
até em um nível de 20%)
""";

                            OLS Regression Results                            
Dep. Variable:                   u_sq   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.117
Date:                Tue, 20 Jun 2023   Prob (F-statistic):              0.291
Time:                        17:12:28   Log-Likelihood:                -10032.
No. Observations:                 706   AIC:                         2.007e+04
Df Residuals:                     704   BIC:                         2.008e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.894e+05   2.05e+04      9.216      0.0

#### 3) Os dados em FERTIL2 (wooldridge) incluem, para mulheres em Botsuana durante 1988, informações sobre o número de filhos, anos de educação, idade e variáveis de status religioso e econômico.

In [44]:
fertil2 = wooldridge.data('fertil2')
fertil2.head()

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr
0,5,64,24,1.0,1.0,1.0,1.0,12,0,NaN,...,NaN,576,1,12,0,0,0,1,0,0
1,1,56,32,1.0,1.0,1.0,1.0,13,3,25.0,...,12.0,1024,1,13,0,0,0,1,0,1
2,7,58,30,1.0,0.0,0.0,0.0,5,1,27.0,...,7.0,900,1,5,1,0,0,0,0,1
3,11,45,42,1.0,0.0,1.0,0.0,4,3,17.0,...,11.0,1764,1,4,0,0,0,0,0,1
4,5,45,43,1.0,1.0,1.0,1.0,11,2,24.0,...,14.0,1849,1,11,0,1,0,1,0,1


##### (i) Estime o modelo abaixo por Mínimos Quadrados Ordinários (OLS) e interprete as estimativas. Em particular, mantendo a idade fixa, qual é o efeito estimado de mais um ano de educação na fertilidade? Se 100 mulheres recebessem mais um ano de educação, quantos filhos a menos elas esperariam ter?

$$\text{children} = \beta_0 + \beta_1 \text{educ} + \beta_2 \text{age} + \beta_3 \text{age}^2 + u$$

In [45]:
formula = 'children ~ educ + age + agesq'
y, X = ps.dmatrices(formula, data=fertil2, return_type='dataframe')
model = sm.OLS(y, X).fit()
print(model.summary())

"""
Mantendo a idade fixa, cada ano de educação a mais resultaria em 0.91 filhos a menos. 
Isso significa que, para um grupo de 100 mulheres de mesma idade que obtiverem mais 
um ano de educação, esperaríamos 9 filhos a menos.
""";

                            OLS Regression Results                            
Dep. Variable:               children   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     1915.
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:12:28   Log-Likelihood:                -7835.6
No. Observations:                4361   AIC:                         1.568e+04
Df Residuals:                    4357   BIC:                         1.570e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.1383      0.241    -17.200      0.0

##### (ii) A variável frsthalf é uma variável dummy que é igual a um se a mulher nasceu durante os primeiros seis meses do ano. Assumindo que frsthalf não está correlacionada com o termo de erro da parte (i), mostre que frsthalf é um candidato razoável para IV (variável instrumental) para educ. (Dica: Você precisa fazer uma regressão.)

In [46]:
# Testamos o potencial de IV usando a forma reduzida de educ.
# Queremos que o coeficiente de frsthalf seja diferente de zero e significante.

formula = 'educ ~ age + agesq + frsthalf'
y, X = ps.dmatrices(formula, data=fertil2, return_type='dataframe')
model = sm.OLS(y, X, missing="drop").fit()
print(model.summary())

"""
Com os resultados, vemos que mulheres que nascem na primeira metade do ano têm uma 
previsão de quase um ano a menos de educação (-0.852), quando a idade é mantida fixa.
A estatística t também é alta em módulo (7.5), o que indica correlação forte com educ.
Finalmente, uma vez que assumimos que frsthalf não é correlacionada ao erro, ela se demonstra
uma boa candidata a IV para educ.
""";

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     175.2
Date:                Tue, 20 Jun 2023   Prob (F-statistic):          3.01e-107
Time:                        17:12:28   Log-Likelihood:                -11905.
No. Observations:                4361   AIC:                         2.382e+04
Df Residuals:                    4357   BIC:                         2.384e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.6929      0.598     16.207      0.0

##### (iii) Estime o modelo da parte (i) usando frsthalf como um IV para educ. Compare o efeito estimado da educação com a estimativa OLS da parte (i).

In [47]:
formula = 'children ~ educ + age + agesq'
y, X = ps.dmatrices(formula, data=fertil2, return_type='dataframe')
df_iv = fertil2[['children', 'educ', 'age', 'agesq', 'frsthalf']].copy()

# Variável dependente
endog = df_iv['children']
# Variáveis independentes (tanto endógenas quanto exógenas)
exog = df_iv[['educ', 'age', 'agesq']]
exog = sm.add_constant(exog)
# Instrumentos (tanto instrumentos quando exógenas)
instr = df_iv[['frsthalf', 'age', 'agesq']]
instr = sm.add_constant(instr)

model = IV2SLS(endog=endog, exog=exog, instrument=instr).fit()
print(model.summary())

"""
O efeito estimado da educação na fertilidade agora é muito maior.
Naturalmente, o erro padrão para a variável educ aumentou, tal como
seu intervalo de confiança.
""";

                          IV2SLS Regression Results                           
Dep. Variable:               children   R-squared:                       0.550
Model:                         IV2SLS   Adj. R-squared:                  0.550
Method:                     Two Stage   F-statistic:                     1765.
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 20 Jun 2023                                         
Time:                        17:12:28                                         
No. Observations:                4361                                         
Df Residuals:                    4357                                         
Df Model:                           3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.3878      0.548     -6.180      0.0

In [48]:
# O Método de Momentos Generalizado chega em resultados semelhantes:
model = IVGMM(endog=endog, exog=exog, instrument=instr).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 11
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 6
         Gradient evaluations: 6
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
                                IVGMM Results                                 
Dep. Variable:               children   Hansen J:                    4.161e-08
Model:                          IVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:

##### (iv) Adicione as variáveis binárias electric, tv e bicycle ao modelo e assuma que essas variáveis são exógenas. Estime a equação por OLS e 2SLS e compare os coeficientes estimados de educ. Interprete o coeficiente de tv e explique por que a posse de televisão tem um efeito negativo na fertilidade.

In [49]:
# Estimando children por OLS
formula = 'children ~ educ + age + agesq + electric + tv + bicycle'
y, X = ps.dmatrices(formula, data=fertil2, return_type='dataframe')
model1 = sm.OLS(y, X, missing="drop").fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:               children   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     984.9
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:12:29   Log-Likelihood:                -7789.3
No. Observations:                4356   AIC:                         1.559e+04
Df Residuals:                    4349   BIC:                         1.564e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.3898      0.240    -18.267      0.0

In [50]:
formula = 'children ~ educ + age + agesq + electric + tv + bicycle'
y, X = ps.dmatrices(formula, data=fertil2, return_type='dataframe')
df_iv = fertil2[['children', 'educ', 'age', 'agesq', 'frsthalf', 'electric', 'tv', 'bicycle']].copy()
df_iv = df_iv.dropna()

# Variável dependente
endog = df_iv['children']
# Variáveis independentes (tanto endógenas quanto exógenas)
exog = df_iv[['educ', 'age', 'agesq', 'electric', 'tv', 'bicycle']]
exog = sm.add_constant(exog)
# Instrumentos (tanto instrumentos quando exógenas)
instr = df_iv[['frsthalf', 'age', 'agesq','electric', 'tv', 'bicycle']]
instr = sm.add_constant(instr)

model2 = IV2SLS(endog=endog, exog=exog, instrument=instr).fit()
print(model2.summary())

                          IV2SLS Regression Results                           
Dep. Variable:               children   R-squared:                       0.558
Model:                         IV2SLS   Adj. R-squared:                  0.557
Method:                     Two Stage   F-statistic:                     921.7
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 20 Jun 2023                                         
Time:                        17:12:29                                         
No. Observations:                4356                                         
Df Residuals:                    4349                                         
Df Model:                           6                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5913      0.645     -5.567      0.0

In [51]:
print('OLS - 1')
print(model.summary().tables[1])
print('OLS - 2')
print(model1.summary().tables[1])
print('IV2SLS')
print(model2.summary().tables[1])

"""
Adicionar eletricidade, televisão e bicicleta ao modelo reduz o efeito estimado 
da educação em ambos os casos, mas não muito. Na equação estimada pelo OLS, 
o coeficiente da televisão implica que, com outros fatores fixos, uma família com
televisão terá -0.25 filhos a menos que uma sem televisão. 

A posse de televisão pode ser um indicador de diferentes coisas, incluindo renda. 
Uma interpretação causal é que a TV fornece uma forma alternativa de recreação. 
Curiosamente, o efeito da posse de TV é estatisticamente insignificante na equação 
estimada por IV (mesmo que não estejamos usando um IV para a TV). 

O coeficiente da eletricidade também é muito reduzido em magnitude na estimativa por IV. 
As quedas desses coeficientes sugerem que um modelo linear pode não ser a 
melhor forma funcional, o que não seria surpreendente, já que o número de filhos 
é uma variável de contagem.
""";

OLS - 1
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.3879      0.545     -6.217      0.000      -4.456      -2.320
educ          -0.1715      0.052     -3.275      0.001      -0.274      -0.069
age            0.3236      0.020     15.998      0.000       0.284       0.363
agesq         -0.0027      0.000     -7.587      0.000      -0.003      -0.002
OLS - 2
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.3898      0.240    -18.267      0.000      -4.861      -3.919
educ          -0.0767      0.006    -12.075      0.000      -0.089      -0.064
age            0.3402      0.016     20.692      0.000       0.308       0.372
agesq         -0.0027      0.000    -10.010      0.000      -0.003      -0.002
electric      -0.3027      0.076    